In [1]:
import os
#Set a specific GPU

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0"  # Set the GPU 0 to use

In [2]:
import torch
import transformers

from transformers import BertTokenizer
from transformers import BertForQuestionAnswering
from transformers import get_cosine_with_hard_restarts_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import re
import pickle
from tqdm.notebook import tqdm

In [3]:
import wandb

wandb.init(project='test_jlk_nlp_qa_n')

wandb: WARNING Path /data5/dep_conv/JLK_NLP/code_qa_2/model_code/wandb/ wasn't writable, using system temp directory.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Path /data5/dep_conv/JLK_NLP/code_qa_2/model_code/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path /data5/dep_conv/JLK_NLP/code_qa_2/model_code/wandb/ wasn't writable, using system temp directory
wandb: Currently logged in as: jun171. Use `wandb login --relogin` to force relogin


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [5]:
main_loc = '../'

with open(main_loc+'data/encoded_data_nl_s_fin.pkl','rb') as f:
    inputs_o, starts_o, ends_o, len_qs_o, masks_o = pickle.load(f)


In [6]:
# inputs_o, _, starts_o, _, ends_o, _, masks_o, _ = \
#     train_test_split(inputs_o, starts_o, ends_o, masks_o, random_state=1, test_size=0.98)

In [7]:
# with open(main_loc+'data/encoded_data_nl_small.pkl','wb') as f:
#     pickle.dump([inputs_o, starts_o, ends_o, masks_o],f)

In [8]:
train_inputs, test_inputs, train_starts, test_starts, train_ends, test_ends, train_masks, test_masks = \
    train_test_split(inputs_o, starts_o, ends_o, masks_o, random_state=1, test_size=0.3)
    
train_inputs, validation_inputs, train_starts, validation_starts, train_ends, validation_ends, train_masks,validation_masks = \
    train_test_split(train_inputs, train_starts, train_ends, train_masks, random_state=0, test_size=0.1)

In [9]:
root_address = main_loc+'model/'

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [11]:
# i = 0
# print(tokenizer.decode(inputs_o[i][starts_o[i]:ends_o[i]+1]))
# print(tokenizer.decode(inputs_o[i][len_qs_o[i][0]:len_qs_o[i][1]]))

In [12]:
train_inputs = torch.tensor(train_inputs)
train_starts = torch.tensor(train_starts)
train_ends = torch.tensor(train_ends)
train_masks = torch.tensor(train_masks)

validation_inputs = torch.tensor(validation_inputs)
validation_starts = torch.tensor(validation_starts)
validation_ends = torch.tensor(validation_ends)
validation_masks = torch.tensor(validation_masks)

test_inputs = torch.tensor(test_inputs)
test_starts = torch.tensor(test_starts)
test_ends = torch.tensor(test_ends)
test_masks = torch.tensor(test_masks)

In [13]:
BATCH_SIZE = 24 #24

train_data = TensorDataset(train_inputs, train_masks, train_starts, train_ends)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_starts, validation_ends)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

test_data = TensorDataset(test_inputs, test_masks, test_starts, test_ends)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [14]:
# model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# model.to(device.type)

In [15]:
model = torch.load(root_address+'model_max_acc.pt')

model.to(device.type)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

In [16]:
optimizer = AdamW(model.parameters(),
                  lr =1e-5, #3e-5
                  eps = 1e-12
                )

epochs = 100


total_steps = len(train_dataloader) * epochs


scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps,
                                            num_cycles=20
                                            )

In [17]:
def cal_dis(f,id):
    rst = 0
    
    if abs(f-id)<5:
        rst+=1/((abs(f-id)+1)**2)
    else:
        pass
    
    return rst

def calc_ans(flats,ids):
    rst = 0
    
    for f,id in zip(flats,ids):
        rst+=cal_dis(f,id)

    return rst

# def calc_ans_bt(st,end,st_ids,end_ids):
#     rst = 0
    
#     for sf,ef,s_id,e_id in zip(st, end, st_ids, end_ids):
#         rst+= cal_dis(sf,s_id)*cal_dis(ef,e_id)
    
#     return rst

def is_same_text(sf,ef,s_id,e_id,input_id):
    re_t = tokenizer.decode(input_id[s_id:e_id+1]).strip()
    ex_t = tokenizer.decode(input_id[sf:ef+1]).strip()
    if re_t == ex_t:    
        return True
    else:
        return False

def calc_ans_bt(st,end,st_ids,end_ids,input_ids):
    rst = 0
    
    for sf,ef,s_id,e_id,input_id in zip(st, end, st_ids, end_ids,input_ids):
        if is_same_text(sf,ef,s_id,e_id,input_id):
            rst+=1

    return rst

def print_expect_val(s_e,e_e,s_i,e_i,input_id):
    re_t = tokenizer.decode(input_id[s_i:e_i+1])
    print(f'Real text: {re_t}')
    if s_e<=e_e:
        ex_t = tokenizer.decode(input_id[s_e:e_e+1])
        print(f'Expect text: {ex_t}')
    else:
        print('No Expect text')
    print()

# def flatten_proper_ans_set(st_s,end_s,n):
#     st_set = []
#     end_set = []
#     for st_p, end_p in zip(st_s,end_s):
#         st = st_p[-n:][0]
#         end = end_p[-n:][0]
#         st_set.append(st)
#         end_set.append(end)
    
#     return st_set,end_set


def flat_accuracy(st_ps,end_ps,st_ids,end_ids,input_ids, pr_t = False,k_n = 3):
    # st_flats = np.argsort(st_ps,kth = -k_n, axis=1)
    # end_flats = np.argsort(end_ps,kth = -k_n, axis=1)
    
    st_flat = np.argmax(st_ps, axis=1).flatten()
    end_flat = np.argmax(end_ps, axis=1).flatten()
    
    # st_acc = np.sum(st_flat == st_ids) / len(st_ids)
    # end_acc = np.sum(end_flat == end_ids) / len(end_ids)    
    # acc = np.sum((st_flat == st_ids) * (end_flat==end_ids)) / len(st_ids)   
    
    if pr_t: #check expected values
        for s_e,e_e,s_i,e_i,input_id in zip(st_flat,end_flat,st_ids,end_ids,input_ids):      
            if is_same_text(s_e,e_e,s_i,e_i,input_id)==False:
                print(f'Expect: {s_e},{e_e}. Real: {s_i},{e_i}')
                print_expect_val(s_e,e_e,s_i,e_i,input_id)
            
    
    st_acc = calc_ans(st_flat, st_ids) / len(st_ids)
    end_acc = calc_ans(end_flat, end_ids) / len(end_ids)
    
    acc = calc_ans_bt(st_flat,end_flat,st_ids,end_ids,input_ids) / len(st_ids)

    return st_acc,end_acc,acc

def format_time(elapsed):

    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [18]:
seed_val = 0
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

max_acc = 0

for epoch_i in range(0, epochs):
    print(f'Training...\n======== Epoch {epoch_i + 1} / {epochs} ========')
    
    t0 = time.time()
    total_loss = 0
    prev_loss = 0
    prev_step = 0
    
    model.train()
    
    for step, batch in enumerate(tqdm(train_dataloader)):

        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            step_loss = (total_loss-prev_loss) / (step-prev_step)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print(f'  Average loss =  {step_loss}')
            print(f"  Last learning rate: {scheduler.get_last_lr()[0]:.3}")
            
            wandb.log({"step loss": step_loss}, step=epoch_i*len(train_dataloader)+step)
            prev_loss = total_loss
            prev_step = step
                        
        if step % 3000 == 0 and not step == 0:
            torch.save(model,root_address+f'model_sv_{epoch_i}_on.pt')
            print(f'model_sv_{epoch_i}_on has been saved')
            

        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_starts, b_ends = batch

        optimizer.zero_grad()

        # Forward         
        outputs = model(b_input_ids, 
                        attention_mask = b_input_mask,
                        start_positions = b_starts,
                        end_positions = b_ends)
        
        st_logits = outputs['start_logits']
        # print('st_log',st_logits)
        # print('st_log_max',max(st_logits[0]))
        end_logits = outputs['end_logits']
        # print('end_log',end_logits)
        
        loss = outputs[0]
        # print('loss', loss)

        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()


    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {:}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
    print(f"  Last learning rate: {scheduler.get_last_lr()[0]:.3}")
    print("")        
    # ========================================
    #               Validation
    # ========================================

    print("Running Validation...")


    t0 = time.time()

    torch.save(model,root_address+f'model_sv_{epoch_i}.pt')
    print(f'model_sv_{epoch_i} has been saved')

    if epoch_i >=2:
        os.remove(root_address+f'model_sv_{epoch_i-2}.pt')
        os.remove(root_address+f'model_sv_{epoch_i-2}_on.pt')
    #Evaluation
    model.eval()

    eval_loss, eval_accuracy1,eval_accuracy2,eval_accuracy3 = 0, 0, 0, 0
    nb_eval_steps = 0

    for batch in tqdm(validation_dataloader):

        batch = tuple(t.to(device) for t in batch)
        

        b_input_ids, b_input_mask, b_starts, b_ends = batch

        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        st_logits = outputs['start_logits']
        end_logits = outputs['end_logits']

        st_logits = st_logits.detach().cpu().numpy()
        end_logits = end_logits.detach().cpu().numpy()
        start_ids = b_starts.to('cpu').numpy()
        end_ids = b_ends.to('cpu').numpy()
        
        
        tmp_eval_accuracy1,tmp_eval_accuracy2,tmp_eval_accuracy3 = flat_accuracy(st_logits,end_logits, start_ids, end_ids,input_ids = b_input_ids)
        eval_accuracy1 += tmp_eval_accuracy1
        eval_accuracy2 += tmp_eval_accuracy2
        eval_accuracy3 += tmp_eval_accuracy3

        nb_eval_steps += 1
        
    acc_all = eval_accuracy3/nb_eval_steps

    print("  st_token Accuracy: {0:.2f}".format(eval_accuracy1/nb_eval_steps))
    print("  end_token Accuracy: {0:.2f}".format(eval_accuracy2/nb_eval_steps))
    print("  all Accuracy: {0:.2f}".format(acc_all))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    
    wandb.log({"val_accuracy": acc_all,"loss": avg_train_loss, "learning_rate": scheduler.get_last_lr()[0]}, step=epoch_i)
    
    if max_acc < acc_all:
        print("")
        print(f"Accuracy_nonzero: {acc_all:0.2f} is higher than previous maximum accuracy: {max_acc:0.2f}")
        max_acc = acc_all
        torch.save(model,root_address+'model_max_acc.pt')
        print(f'New model_max_acc has been saved')    
    
    print("")

    # ========================================
    #                  Test
    # ========================================
    if epoch_i%5 ==0 and epoch_i !=0:    
        print("Running Test...")

        t0 = time.time()
            
        model.eval()

        eval_loss, eval_accuracy1,eval_accuracy2,eval_accuracy3 = 0, 0, 0, 0
        nb_eval_steps = 0

        for batch in tqdm(test_dataloader):

            batch = tuple(t.to(device) for t in batch)

            b_input_ids, b_input_mask, b_starts, b_ends = batch

            with torch.no_grad():     
                outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask)
            
            st_logits = outputs['start_logits']
            end_logits = outputs['end_logits']

            st_logits = st_logits.detach().cpu().numpy()
            end_logits = end_logits.detach().cpu().numpy()
            start_ids = b_starts.to('cpu').numpy()
            end_ids = b_ends.to('cpu').numpy()
            
            
            tmp_eval_accuracy1,tmp_eval_accuracy2,tmp_eval_accuracy3 = flat_accuracy(st_logits,end_logits, start_ids, end_ids, input_ids=b_input_ids)
            eval_accuracy1 += tmp_eval_accuracy1
            eval_accuracy2 += tmp_eval_accuracy2
            eval_accuracy3 += tmp_eval_accuracy3

            nb_eval_steps += 1
            
        acc_all = eval_accuracy3/nb_eval_steps
        
        wandb.log({"test_accuracy": acc_all}, step=epoch_i)
        
        print("  st_token Accuracy: {0:.2f}".format(eval_accuracy1/nb_eval_steps))
        print("  end_token Accuracy: {0:.2f}".format(eval_accuracy2/nb_eval_steps))
        print("  all Accuracy: {0:.2f}".format(acc_all))
        print("  Test took: {:}".format(format_time(time.time() - t0)))

        print("")
    
print("Training complete!")

torch.save(model,root_address+'model_fin.pt')

Training...
======== Epoch 1 / 100 ========


  0%|          | 0/5641 [00:00<?, ?it/s]

  Batch   500  of  5,641.    Elapsed: 0:14:11.
  Average loss =  0.7725441969037056
  Last learning rate: 9.99e-06
  Batch 1,000  of  5,641.    Elapsed: 0:28:18.
  Average loss =  0.7500127480626106
  Last learning rate: 9.97e-06
  Batch 1,500  of  5,641.    Elapsed: 0:42:26.
  Average loss =  0.7436734752058983
  Last learning rate: 9.93e-06
  Batch 2,000  of  5,641.    Elapsed: 0:56:34.
  Average loss =  0.7672943090200424
  Last learning rate: 9.88e-06
  Batch 2,500  of  5,641.    Elapsed: 1:10:43.
  Average loss =  0.7592478284835815
  Last learning rate: 9.81e-06
  Batch 3,000  of  5,641.    Elapsed: 1:24:52.
  Average loss =  0.7728655940592289
  Last learning rate: 9.72e-06
model_sv_0_on has been saved
  Batch 3,500  of  5,641.    Elapsed: 1:39:57.
  Average loss =  0.7638937972187996
  Last learning rate: 9.62e-06
  Batch 4,000  of  5,641.    Elapsed: 1:54:07.
  Average loss =  0.7839522058963776
  Last learning rate: 9.51e-06
  Batch 4,500  of  5,641.    Elapsed: 2:08:18.
  Av

  0%|          | 0/627 [00:00<?, ?it/s]

  st_token Accuracy: 0.60
  end_token Accuracy: 0.60
  all Accuracy: 0.58
  Validation took: 0:06:22

Accuracy_nonzero: 0.58 is higher than previous maximum accuracy: 0.00
New model_max_acc has been saved

Training...
======== Epoch 2 / 100 ========


...

  0%|          | 0/627 [00:00<?, ?it/s]

  st_token Accuracy: 0.59
  end_token Accuracy: 0.59
  all Accuracy: 0.57
  Validation took: 0:06:32

Training...
======== Epoch 18 / 100 ========


  0%|          | 0/5641 [00:00<?, ?it/s]

  Batch   500  of  5,641.    Elapsed: 0:14:03.
  Average loss =  0.7649317989945412
  Last learning rate: 6.28e-06
  Batch 1,000  of  5,641.    Elapsed: 0:28:05.
  Average loss =  0.7499041575491429
  Last learning rate: 6.01e-06
  Batch 1,500  of  5,641.    Elapsed: 0:42:08.
  Average loss =  0.7675535350143909
  Last learning rate: 5.73e-06
  Batch 2,000  of  5,641.    Elapsed: 0:56:10.
  Average loss =  0.7461438678205013
  Last learning rate: 5.46e-06
  Batch 2,500  of  5,641.    Elapsed: 1:10:12.
  Average loss =  0.7546912826895714
  Last learning rate: 5.18e-06


KeyboardInterrupt: 

In [ ]:
# ========================================
#                  Test
# ========================================
print("Running Test...")

t0 = time.time()
    
model.eval()

eval_loss, eval_accuracy1,eval_accuracy2,eval_accuracy3 = 0, 0, 0, 0
nb_eval_steps = 0

for batch in tqdm(test_dataloader):

    batch = tuple(t.to(device) for t in batch)
    

    b_input_ids, b_input_mask, b_starts, b_ends = batch

    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    st_logits = outputs['start_logits']
    end_logits = outputs['end_logits']

    st_logits = st_logits.detach().cpu().numpy()
    end_logits = end_logits.detach().cpu().numpy()
    start_ids = b_starts.to('cpu').numpy()
    end_ids = b_ends.to('cpu').numpy()
    
    
    tmp_eval_accuracy1,tmp_eval_accuracy2,tmp_eval_accuracy3 = flat_accuracy(st_logits,end_logits, start_ids, end_ids, input_ids=b_input_ids)
    eval_accuracy1 += tmp_eval_accuracy1
    eval_accuracy2 += tmp_eval_accuracy2
    eval_accuracy3 += tmp_eval_accuracy3

    nb_eval_steps += 1
    
acc_all = eval_accuracy3/nb_eval_steps
wandb.log({"End_test_accuracy": acc_all})

print("  st_token Accuracy: {0:.2f}".format(eval_accuracy1/nb_eval_steps))
print("  end_token Accuracy: {0:.2f}".format(eval_accuracy2/nb_eval_steps))
print("  all Accuracy: {0:.2f}".format(acc_all))
print("  Test took: {:}".format(format_time(time.time() - t0)))

print("")